In [22]:
import pdfplumber
def extract_text_from_pdf(pdf_path):
    all_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                all_text += text + "\n\n\n\n"
    return all_text

In [144]:
# Path to your PDF file
pdf_path = 'data/pdf4.pdf'

# Extract text from the PDF
extracted_text = extract_text_from_pdf(pdf_path)
print(extracted_text)

学习资料 仅供参考
认真学好用好《准则》《条例》
把党规党纪刻印在全体党员心上
西北工业大学纪委办公室
2015年11月5日



2015年10月18日
中共中央颁布实施新
修订的《中国共产党廉洁
自律准则》和《中国共产
党纪律处分条例》。
2016年1月1日
《准则》和《条例》的颁布实施，
《中国共产党廉洁自律
是在党长期执政和全面依法治国条件
准则》和《中国共产党纪律
下，实现依规管党治党、加强党内监
处分条例》将正式施行。 督的重大举措，体现了党的十八大和
十八届三中、四中全会精神以及全面
从严治党实践成果。



近日，中共中央办公厅印发了《关于认真学习
贯彻〈中国共产党廉洁自律准则〉和〈中国共产党
纪律处分条例〉的通知》，要求：
充分认识学习贯彻《准则》《条例》的重要意义
认真组织《准则》和《条例》的学习宣传教育
切实抓好《准则》和《条例》的贯彻实施



10月29日，中央纪委印发《通知》，要求：
把学习贯彻《准则》和《条例》作为落实全面从严治
党要求的重要抓手
把学习贯彻《准则》和《条例》体现到监督执纪问责
全过程
加强对学习贯彻《准则》和《条例》情况的监督检查
自觉做学习贯彻《准则》和《条例》的表率



一、 学习《准则》、《条例》
的意义
二、《中国共产党廉洁自律
准则》修订情况
三、《中国共产党纪律处分
学习内容
条例》修订情况
四、 《准则》、《条例》
的关系
五、 几点要求



（一）体制机制层面题
治国理政的要求：办好中国的事情，关键在党。治国
必先治党，治党务必从严。当前我们面临着党的领导
弱化和组织涣散、纪律松弛的严峻挑战。加强纪律建
设是全面从严治党的治本之策。
时代发展的要求：新修订的《准则》和《条例》，把
十八大以来从严治党的实践成果总结提炼出来，转化
为道德和纪律要求，实现了党内法规建设的与时俱进。



（一）体制机制层面题
从严治党的要求：全面从严治党，必然要求依规治党
和以德治党相结合。新修订的《准则》和《条例》体
现了依规治党和以德治党的统一。
依法治校的要求：深入学习贯彻落实两部党内法规，
完善学校治理体系，提升学校治理能力，为学校制定
实施“十三五”发展规划和学校长远发展保驾护航。



一、 学习《准则》、《条
例》
的意义
二、《中国共产党廉洁自律
准则》修订情况
三、《中国共产党纪律处分
学

In [145]:
# from langchain import PromptTemplate
from langchain_groq import ChatGroq
# from langchain.chains.summarize import load_summarize_chain
# from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv
load_dotenv()

## Load the Groq API
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
# summarization_model = ChatGroq(model_name = "llama-3.1-70b-versatile")
summarization_model = ChatGroq(model_name = "llama-3.1-8b-instant")


In [146]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=20)
# chunks = text_splitter.create_documents([extracted_text])

In [147]:
# len(chunks)

In [152]:
from deep_translator import GoogleTranslator
from transformers import pipeline
import textwrap

# Load the summarization model
# summarization_model = pipeline("summarization", model="facebook/bart-large-cnn")

def translate_text(text):
    """Translate Chinese text to English using Deep Translator."""
    try:
        translated_text = GoogleTranslator(source='zh-CN', target='en').translate(text)
        return translated_text
    except Exception as e:
        print(f"Error during translation: {e}")
        return ""

def summarize_text(text):
    text = f"Summarize the following text in 50-60 words only. '{text}'"
    """Summarize English text."""
    try:
        # Check if text is not empty
        if len(text) > 0:
            # Ensure the text is within the summarization model's limits
            summary = summarization_model.invoke(text)
            return summary
        else:
            print("No text to summarize.")
            return ""
    except Exception as e:
        print(f"Error during summarization: {e}")
        return ""

def process_long_text(long_text, chunk_size=1000):
    """Process a long text: translate and summarize."""
    # Split the long text into manageable chunks
    chunks = textwrap.wrap(long_text, width=chunk_size, break_long_words=False)
    
    translated_chunks = []
    for chunk in chunks:
        translated_chunk = translate_text(chunk)
        if translated_chunk:  # Ensure translated chunk is not empty
            translated_chunks.append(translated_chunk)
    
    # Combine all translated chunks
    combined_translated_text = " ".join(translated_chunks)
    print(combined_translated_text)

    # Only summarize if we have valid translated text
    if combined_translated_text.strip():  # Check if there's any valid translated text
        summary = summarize_text(combined_translated_text)
        return summary
    else:
        return "No valid translated text available for summarization."

In [153]:
long_chinese_text = extracted_text # Your long Chinese text here
print(long_chinese_text)

学习资料 仅供参考
认真学好用好《准则》《条例》
把党规党纪刻印在全体党员心上
西北工业大学纪委办公室
2015年11月5日



2015年10月18日
中共中央颁布实施新
修订的《中国共产党廉洁
自律准则》和《中国共产
党纪律处分条例》。
2016年1月1日
《准则》和《条例》的颁布实施，
《中国共产党廉洁自律
是在党长期执政和全面依法治国条件
准则》和《中国共产党纪律
下，实现依规管党治党、加强党内监
处分条例》将正式施行。 督的重大举措，体现了党的十八大和
十八届三中、四中全会精神以及全面
从严治党实践成果。



近日，中共中央办公厅印发了《关于认真学习
贯彻〈中国共产党廉洁自律准则〉和〈中国共产党
纪律处分条例〉的通知》，要求：
充分认识学习贯彻《准则》《条例》的重要意义
认真组织《准则》和《条例》的学习宣传教育
切实抓好《准则》和《条例》的贯彻实施



10月29日，中央纪委印发《通知》，要求：
把学习贯彻《准则》和《条例》作为落实全面从严治
党要求的重要抓手
把学习贯彻《准则》和《条例》体现到监督执纪问责
全过程
加强对学习贯彻《准则》和《条例》情况的监督检查
自觉做学习贯彻《准则》和《条例》的表率



一、 学习《准则》、《条例》
的意义
二、《中国共产党廉洁自律
准则》修订情况
三、《中国共产党纪律处分
学习内容
条例》修订情况
四、 《准则》、《条例》
的关系
五、 几点要求



（一）体制机制层面题
治国理政的要求：办好中国的事情，关键在党。治国
必先治党，治党务必从严。当前我们面临着党的领导
弱化和组织涣散、纪律松弛的严峻挑战。加强纪律建
设是全面从严治党的治本之策。
时代发展的要求：新修订的《准则》和《条例》，把
十八大以来从严治党的实践成果总结提炼出来，转化
为道德和纪律要求，实现了党内法规建设的与时俱进。



（一）体制机制层面题
从严治党的要求：全面从严治党，必然要求依规治党
和以德治党相结合。新修订的《准则》和《条例》体
现了依规治党和以德治党的统一。
依法治校的要求：深入学习贯彻落实两部党内法规，
完善学校治理体系，提升学校治理能力，为学校制定
实施“十三五”发展规划和学校长远发展保驾护航。



一、 学习《准则》、《条
例》
的意义
二、《中国共产党廉洁自律
准则》修订情况
三、《中国共产党纪律处分
学

In [154]:
# Process the long text
english_summary = process_long_text(long_chinese_text)

Study materials For reference only Study and use the Code and Regulations carefully and make good use of them Engrave the Party rules and disciplines in the hearts of all Party members Office of the Discipline Inspection Commission of Northwestern Polytechnical University November 5, 2015 On October 18, 2015, the CPC Central Committee promulgated and implemented the newly revised Code of Conduct for Integrity and Self-discipline of the Communist Party of China and the Regulations on Disciplinary Punishment of the Communist Party of China. On January 1, 2016, the promulgation and implementation of the Code and Regulations, the Code of Conduct for Integrity and Self-discipline of the Communist Party of China and the Regulations on Disciplinary Punishment of the Communist Party of China will be formally implemented. It is a major measure to achieve the management and governance of the Party in accordance with regulations and strengthen the supervision within the Party under the conditions

In [155]:
from pprint import pprint
pprint(english_summary.content)

("Here's a summary of the text in 50-60 words:\n"
 '\n'
 'The Communist Party of China has revised its Code of Conduct for Integrity '
 'and Self-discipline and Regulations on Disciplinary Punishment to strengthen '
 'party governance and discipline. The revisions emphasize honesty, '
 'self-discipline, and party spirit, and provide clearer guidelines for party '
 'members and leading cadres to adhere to.')


MAP REDUCE

In [201]:
from langchain.document_loaders import PyPDFLoader
pdf_file = "data/pdf1.pdf"
pdf_loader = PyPDFLoader(pdf_file)
pages = pdf_loader.load_and_split()
pages

[Document(metadata={'source': 'data/pdf1.pdf', 'page': 0}, page_content='技术性贸易措施体系建设项目  CS03-13 \n \n \n \n各国化妆品标签管理指南  \n \n \n \n \n \n \n中国检验检疫科学研究院  \n \n二〇一二年十月'),
 Document(metadata={'source': 'data/pdf1.pdf', 'page': 1}, page_content='使用说明：  \n     \n1、本《出口商品技术指南》将至少半年更新一次； \n    2、本《出口商品技术指南》电子文本使用 PDF 格式，浏览须安装Adobe 公司免费提供的Adobe \nAcrobat软件。简体中文版可点击 Adobe Reader 6.0 下载。 \n  3、用户可在线浏览，或将 PDF 文件下载到本地机器后阅读。   4、如有疑问或意见建议请与商务部世贸司联系，电子邮件：\ndstdiv3@mofcom.gov.cn\n   \n版权声明： \n  《出口商品技术指南》版权归中华人民共和国商务部所有，供公众免费查阅。未经商务部授权，任何单位或个人不得将其用于任何商业盈利目的，不得转载、摘编、变更或出版《出口商品技术指南》。经商务部授权的，应在授权范围内使用，并注明“来源：中华人民共和国商务部”。违反上述声明者，商务部将追究其相关法律责任。'),
 Document(metadata={'source': 'data/pdf1.pdf', 'page': 2}, page_content='I \n 目   录 \n一、适用范围 ................................ ................................ ................................ ................  1 \n二、进出口化妆品基本情况概述 ................................ ................................ ................  2 \n2.1 术语和定义  ................................ ...

In [202]:
from langchain.prompts import PromptTemplate
map_prompt_template = """
Write a summary of this chunk of text that includes the main points and any important details. just write in 40-50 words.
{text}
"""

main_prompt = PromptTemplate(template = map_prompt_template, input_variables=["text"])

combined_prompt_template = """
Write a concise summary of the following text delimited by triple backquotes.
Return your response in bullet points which covers the key points of the text.
```{text}```
BULLET POINT SUMMARY:
"""

combine_prompt = PromptTemplate(
    template=combined_prompt_template, input_variables=["text"]
)

In [203]:
# from langchain import PromptTemplate
from langchain_groq import ChatGroq
# from langchain.chains.summarize import load_summarize_chain
# from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv
load_dotenv()

## Load the Groq API
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
summarization_model = ChatGroq(model_name = "llama-3.1-70b-versatile")
# summarization_model = ChatGroq(model_name = "llama-3.1-8b-instant")

In [204]:
from langchain.chains import load_summarize_chain
map_reduce_cahin = load_summarize_chain(
    summarization_model,
    chain_type="map_reduce",
    combine_prompt=combine_prompt,
    return_intermediate_steps = True,    
)

In [205]:
from deep_translator import GoogleTranslator 
from langchain_core.documents import Document
# Function to translate and reformat documents
def translate_documents(documents):
    translated_docs = []
    
    for doc in documents:
        # Translate the content
        translated_content = GoogleTranslator(source='zh-CN', target='en').translate(doc.page_content)
        
        # Create a new Document with the translated content
        translated_doc = Document(metadata=doc.metadata, page_content=translated_content)
        
        # Append the new Document to the list
        translated_docs.append(translated_doc)
    
    return translated_docs
translated_docs = translate_documents(pages)

In [206]:
translated_docs

[Document(metadata={'source': 'data/pdf1.pdf', 'page': 0}, page_content='Technical Trade Measures System Construction Project CS03-13 \n\nGuidelines for Cosmetic Label Management in Various Countries \n\nChina Institute of Inspection and Quarantine \n\nOctober 2012'),
 Document(metadata={'source': 'data/pdf1.pdf', 'page': 1}, page_content='Instructions for use: \n\n1. This "Technical Guide for Export Commodities" will be updated at least once every six months; \n2. The electronic text of this "Technical Guide for Export Commodities" uses PDF format. Adobe \nAcrobat software provided free of charge by Adobe must be installed to browse it. The simplified Chinese version can be downloaded by clicking Adobe Reader 6.0. \n3. Users can browse online or download the PDF file to a local machine for reading. 4. If you have any questions or comments, please contact the WTO Department of the Ministry of Commerce, email: \ndstdiv3@mofcom.gov.cn\n\nCopyright Statement: \nThe copyright of the "Techn

In [231]:
len(translated_docs)

145

GENERATE PDF OF TRANSLATED TEXT

In [236]:
from fpdf import FPDF
FONT_SIZE = 8
FONT_STYLE = "Arial"
MARGIN = 10
LINE_SPACE = 5
PDF_NAME = "multi_page_document.pdf"

# Create a PDF object
pdf = FPDF(orientation='P', unit='mm', format='A4') # P:portrait, L:Landscape, 'mm': Millimeters (default), 'pt': Points (1 point = 1/72 of an inch), 'cm': Centimeters, 'in': Inches
pdf.set_auto_page_break(auto=True, margin=MARGIN)
pdf.set_font(FONT_STYLE, size=FONT_SIZE)

# Loop through each Document object and add its content to a new page
for doc in translated_docs:
    pdf.add_page()
    pdf.multi_cell(0, LINE_SPACE, doc.page_content.encode('latin-1', 'replace').decode('latin-1'))  # Encode with 'replace' to handle unsupported chars

# Save the PDF file
pdf_filename = PDF_NAME
pdf.output(pdf_filename)

print(f"PDF '{pdf_filename}' created successfully.")

PDF 'multi_page_document.pdf' created successfully.


In [207]:
# map_reduce_output = map_reduce_cahin({"input_documents":pages})
# map_reduce_output

In [208]:
translated_docs[0:5]

[Document(metadata={'source': 'data/pdf1.pdf', 'page': 0}, page_content='Technical Trade Measures System Construction Project CS03-13 \n\nGuidelines for Cosmetic Label Management in Various Countries \n\nChina Institute of Inspection and Quarantine \n\nOctober 2012'),
 Document(metadata={'source': 'data/pdf1.pdf', 'page': 1}, page_content='Instructions for use: \n\n1. This "Technical Guide for Export Commodities" will be updated at least once every six months; \n2. The electronic text of this "Technical Guide for Export Commodities" uses PDF format. Adobe \nAcrobat software provided free of charge by Adobe must be installed to browse it. The simplified Chinese version can be downloaded by clicking Adobe Reader 6.0. \n3. Users can browse online or download the PDF file to a local machine for reading. 4. If you have any questions or comments, please contact the WTO Department of the Ministry of Commerce, email: \ndstdiv3@mofcom.gov.cn\n\nCopyright Statement: \nThe copyright of the "Techn

In [209]:
response = map_reduce_cahin({"input_documents":translated_docs})
response['output_text']

"**Cosmetics Industry Overview**\n\n* China's cosmetics industry has experienced significant growth, with exports increasing by 175.51% and imports by 103.50% from 2007 to 2011.\n* The industry has competitive advantages in the international market, particularly in price and technology.\n* China's unique resources of Chinese herbal medicines and its growing related industries have driven the development of the cosmetics industry.\n\n**Export and Import Trends**\n\n* Skin, hair, oral, and beauty cosmetics account for over 80% of the total export value, with skin cosmetics experiencing the highest growth rate.\n* China's main cosmetics export regions are Asia, America, and Europe, with varying growth rates among these regions.\n* The top 10 countries/regions accounted for 60.79% of the quantity and 69.70% of the total value of cosmetics exports in 2011.\n\n**Labeling Regulations and Requirements**\n\n* Cosmetic labeling regulations vary by country, with the EU, US, Japan, and South Korea

In [210]:
print(response['output_text'])

**Cosmetics Industry Overview**

* China's cosmetics industry has experienced significant growth, with exports increasing by 175.51% and imports by 103.50% from 2007 to 2011.
* The industry has competitive advantages in the international market, particularly in price and technology.
* China's unique resources of Chinese herbal medicines and its growing related industries have driven the development of the cosmetics industry.

**Export and Import Trends**

* Skin, hair, oral, and beauty cosmetics account for over 80% of the total export value, with skin cosmetics experiencing the highest growth rate.
* China's main cosmetics export regions are Asia, America, and Europe, with varying growth rates among these regions.
* The top 10 countries/regions accounted for 60.79% of the quantity and 69.70% of the total value of cosmetics exports in 2011.

**Labeling Regulations and Requirements**

* Cosmetic labeling regulations vary by country, with the EU, US, Japan, and South Korea having specifi

In [212]:
from pathlib import Path
final_mp_data = []
for doc, out in zip(response["input_documents"], response["intermediate_steps"]):
    output = {}
    output["file_name"] = Path(doc.metadata["source"]).stem
    output["file_type"] = Path(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["concise_summary"] = out
    final_mp_data.append(output)

In [213]:
import pandas as pd
pdf_mp_summary = pd.DataFrame.from_dict(final_mp_data)
pdf_mp_summary = pdf_mp_summary.sort_values(by=["file_name", "page_number"])
pdf_mp_summary.reset_index(inplace=True, drop=True)
pdf_mp_summary.head()

,file_name,file_type,page_number,chunks,concise_summary
0,pdf1,.pdf,0,Technical Trade Measures System Construction P...,The China Institute of Inspection and Quaranti...
1,pdf1,.pdf,1,"Instructions for use: \n\n1. This ""Technical G...","The ""Technical Guide for Export Commodities"" i..."
2,pdf1,.pdf,2,I \nTable of Contents \nI. Scope of Applicatio...,This report provides an overview of China's co...
3,pdf1,.pdf,3,II 3.1.1 Legal documents ........................,This document outlines the regulations and gui...
4,pdf1,.pdf,4,III \n3.6.2 Cosmetics labeling requirements in...,The document discusses cosmetics labeling requ...


PAGE WISE SUMMARY

In [215]:
index = 1
print("[Context]")
print(pdf_mp_summary["chunks"].iloc[index])
print("\n\n[Simple Summary]")
print(pdf_mp_summary["concise_summary"].iloc[index])
print("\n\n[Page number]")
print(pdf_mp_summary["page_number"].iloc[index])
print("\n\n[Source: file_name]")
print(pdf_mp_summary["file_name"].iloc[index])

[Context]
Instructions for use: 

1. This "Technical Guide for Export Commodities" will be updated at least once every six months; 
2. The electronic text of this "Technical Guide for Export Commodities" uses PDF format. Adobe 
Acrobat software provided free of charge by Adobe must be installed to browse it. The simplified Chinese version can be downloaded by clicking Adobe Reader 6.0. 
3. Users can browse online or download the PDF file to a local machine for reading. 4. If you have any questions or comments, please contact the WTO Department of the Ministry of Commerce, email: 
dstdiv3@mofcom.gov.cn

Copyright Statement: 
The copyright of the "Technical Guide for Export Commodities" belongs to the Ministry of Commerce of the People's Republic of China and is available to the public free of charge. Without the authorization of the Ministry of Commerce, no unit or individual may use it for any commercial profit purpose, and may not reprint, excerpt, change or publish the "Technical Gui

PPT or PPTX READER

In [247]:
from langchain_community.document_loaders import UnstructuredPowerPointLoader

def extract_text_from_ppt_pagewise(ppt_path):
    # Load the PowerPoint file in elements mode
    loader = UnstructuredPowerPointLoader(ppt_path, mode="elements")
    documents = loader.load()

    # Dictionary to hold text grouped by page number
    pagewise_text = {}

    # Group text based on page_number
    for doc in documents:
        page_number = doc.metadata.get("page_number", None)

        # Skip if page_number is None or replace with a default page number (e.g., 0)
        if page_number is None or not doc.page_content.strip():
            continue  # You can use a default value or continue to skip None values.

        # Initialize the key if not already present and concatenate text
        if page_number not in pagewise_text:
            pagewise_text[page_number] = doc.page_content
        else:
            pagewise_text[page_number] += f" {doc.page_content}"  # Concatenate with a space for better readability

    # Convert the grouped text to a list, sorted by page number
    pages_text = [text for _, text in sorted(pagewise_text.items())]

    return pages_text

# Example usage
# ppt_path = "data/20151014180939084157.ppt"
ppt_path = "data/20151110.pptx"
page_text_list = extract_text_from_ppt_pagewise(ppt_path)

# Print the concatenated text for each page
for i, page_text in enumerate(page_text_list, 1):
    print(f"Page {i}: {page_text}")

Page 1: 学习资料  仅供参考 认真学好用好《准则》《条例》 把党规党纪刻印在全体党员心上 西北工业大学纪委办公室 2015年11月5日
Page 2: 把党规党纪刻印在全体党员心上 2015年10月18日        中共中央颁布实施新修订的《中国共产党廉洁自律准则》和《中国共产党纪律处分条例》。 2016年1月1日       《准则》和《条例》的颁布实施，是在党长期执政和全面依法治国条件下，实现依规管党治党、加强党内监督的重大举措，体现了党的十八大和十八届三中、四中全会精神以及全面从严治党实践成果。     《中国共产党廉洁自律准则》和《中国共产党纪律处分条例》将正式施行。
Page 3: 把党规党纪刻印在全体党员心上        近日，中共中央办公厅印发了《关于认真学习贯彻〈中国共产党廉洁自律准则〉和〈中国共产党纪律处分条例〉的通知》，要求： 充分认识学习贯彻《准则》《条例》的重要意义 认真组织《准则》和《条例》的学习宣传教育 切实抓好《准则》和《条例》的贯彻实施
Page 4: 把党规党纪刻印在全体党员心上 10月29日，中央纪委印发《通知》，要求： 把学习贯彻《准则》和《条例》作为落实全面从严治党要求的重要抓手 把学习贯彻《准则》和《条例》体现到监督执纪问责全过程 加强对学习贯彻《准则》和《条例》情况的监督检查 自觉做学习贯彻《准则》和《条例》的表率
Page 5:    一、 学习《准则》、《条例》                              的意义    二、《中国共产党廉洁自律              准则》修订情况    三、《中国共产党纪律处分             条例》修订情况 学习内容    四、 《准则》、《条例》             的关系    五、 几点要求
Page 6: （一）体制机制层面题   一、学习《准则》、《条例》的意义 治国理政的要求：办好中国的事情，关键在党。治国必先治党，治党务必从严。当前我们面临着党的领导弱化和组织涣散、纪律松弛的严峻挑战。加强纪律建设是全面从严治党的治本之策。 时代发展的要求：新修订的《准则》和《条例》，把十八大以来从严治党的实践成果总结提炼出来，转化为道德和纪律要求，实现了党内法规建设的与时俱进。
Page 7: （一）体制机制层面题   一、学习《准则》、《

REFINE 

In [220]:
from langchain.prompts import PromptTemplate

question_prompt_template = """
Please provide a summary of the following text.
TEXT: {text}
SUMMARY:
"""

question_prompt = PromptTemplate(template = question_prompt_template, input_variables=["text"])

refine_prompt_template = """
Write a concise summary of the following text delimited by triple backquotes.
Return your response in bullet points which covers the key points of the text.
```{text}```
BULLET POINT SUMMARY:
"""

refine_prompt = PromptTemplate(
    template=refine_prompt_template, input_variables=["text"]
)

In [221]:
refine_chain = load_summarize_chain(
    summarization_model,
    chain_type="refine",
    question_prompt=question_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps = True,
)

In [222]:
refine_outputs = refine_chain({"input_documents": translated_docs})

In [227]:
print(refine_outputs['output_text'])

* List of source institutions for cosmetics regulations and industry information in various countries or regions.
* Countries or regions covered include China, Europe, the United States, Japan, Korea, Taiwan, Saudi Arabia, Canada, Russia, and Australia.
* Key institutions include government ministries (e.g., Ministry of Commerce of China, Ministry of Health, Labor and Welfare of Japan), regulatory agencies (e.g., U.S. FDA, Health Canada), and industry associations (e.g., China Association of Flavors and Fragrances, European Cosmetics, Toiletries and Fragrances Association).
* Websites for these institutions provide access to cosmetics regulations, industry information, and other relevant data.


In [223]:
from pathlib import Path
final_refine_data = []
for doc, out in zip(refine_outputs["input_documents"], refine_outputs["intermediate_steps"]):
    output = {}
    output["file_name"] = Path(doc.metadata["source"]).stem
    output["file_type"] = Path(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["concise_summary"] = out
    final_refine_data.append(output)

In [224]:
import pandas as pd
pdf_mp_summary = pd.DataFrame.from_dict(final_refine_data)
pdf_mp_summary = pdf_mp_summary.sort_values(by=["file_name", "page_number"])
pdf_mp_summary.reset_index(inplace=True, drop=True)
pdf_mp_summary.head()

,file_name,file_type,page_number,chunks,concise_summary
0,pdf1,.pdf,0,Technical Trade Measures System Construction P...,"Unfortunately, you haven't provided the actual..."
1,pdf1,.pdf,1,"Instructions for use: \n\n1. This ""Technical G...","* The ""Technical Guide for Export Commodities""..."
2,pdf1,.pdf,2,I \nTable of Contents \nI. Scope of Applicatio...,**Key Points:**\n\n* The text is a report on t...
3,pdf1,.pdf,3,II 3.1.1 Legal documents ........................,BULLET POINT SUMMARY:\n\n* The text appears to...
4,pdf1,.pdf,4,III \n3.6.2 Cosmetics labeling requirements in...,* The text appears to be a part of a report or...


In [225]:
index = 1
print("[Context]")
print(pdf_mp_summary["chunks"].iloc[index])
print("\n\n[Simple Summary]")
print(pdf_mp_summary["concise_summary"].iloc[index])
print("\n\n[Page number]")
print(pdf_mp_summary["page_number"].iloc[index])
print("\n\n[Source: file_name]")
print(pdf_mp_summary["file_name"].iloc[index])

[Context]
Instructions for use: 

1. This "Technical Guide for Export Commodities" will be updated at least once every six months; 
2. The electronic text of this "Technical Guide for Export Commodities" uses PDF format. Adobe 
Acrobat software provided free of charge by Adobe must be installed to browse it. The simplified Chinese version can be downloaded by clicking Adobe Reader 6.0. 
3. Users can browse online or download the PDF file to a local machine for reading. 4. If you have any questions or comments, please contact the WTO Department of the Ministry of Commerce, email: 
dstdiv3@mofcom.gov.cn

Copyright Statement: 
The copyright of the "Technical Guide for Export Commodities" belongs to the Ministry of Commerce of the People's Republic of China and is available to the public free of charge. Without the authorization of the Ministry of Commerce, no unit or individual may use it for any commercial profit purpose, and may not reprint, excerpt, change or publish the "Technical Gui